In [2]:
import datalab.bigquery as bq
import tensorflow as tf
import pandas as pd
import numpy as np
import shutil

/usr/local/envs/py2env/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
columns=['fare_amount', 'pickuplon','pickuplat','dropofflon','dropofflat','passengers', 'key']

In [4]:
features=['pickuplon','pickuplat','dropofflon','dropofflat','passengers', 'key']

In [23]:
label=['fare_amount']

In [6]:
df_train=pd.read_csv('./taxi-train.csv', header = None, names = columns)

In [14]:
df_train.head()

,fare_amount,pickuplon,pickuplat,dropofflon,dropofflat,passengers,key
0,12.0,-73.987625,40.750617,-73.971163,40.785180,1,0
1,4.5,-73.963620,40.774363,-73.953485,40.772665,1,1
2,4.5,-73.989649,40.756633,-73.985597,40.765662,1,2
3,10.0,-73.993950,40.727524,-74.006584,40.744240,1,3
4,2.5,-73.950223,40.668960,-73.948112,40.668872,6,4


In [7]:
df_valid=pd.read_csv('./taxi-valid.csv', header = None, names = columns)

In [8]:
df_test=pd.read_csv('./taxi-test.csv', header = None, names = columns)

In [10]:
# Create input function to read data

In [25]:
 def input_fn(df):
    return tf.estimator.inputs.pandas_input_fn(
    x=df,
    y=df[label],
    batch_size=128,
    num_epochs=10,
    shuffle=True)

In [ ]:
# Create predict function to read data

In [32]:
 def predict_fn(df):
    return tf.estimator.inputs.pandas_input_fn(
    x=df,
    batch_size=128,
    num_epochs=10,
    shuffle=True)

In [13]:
# Create feature columns for estimator

In [15]:
 features_col =[tf.feature_column.numeric_column(a) for a in features]


In [19]:
 features_col

[_NumericColumn(key='pickuplon', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='pickuplat', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='dropofflon', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='dropofflat', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='passengers', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='key', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

In [16]:
# Linear Regression with tf.Estimator framework

In [18]:
tf.logging.set_verbosity(tf.logging.INFO)

OUTDIR = 'taxi_trained'
shutil.rmtree(OUTDIR, ignore_errors = True) # start fresh each time

model=tf.estimator.LinearRegressor(features_col,model_dir = OUTDIR)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_train_distribute': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fac57f38e50>, '_evaluation_master': '', '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': 'taxi_trained', '_global_id_in_cluster': 0, '_save_summary_steps': 100}


In [26]:
model.train(input_fn(df_train)) 

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into taxi_trained/model.ckpt.
INFO:tensorflow:loss = 33514.414, step = 1
INFO:tensorflow:global_step/sec: 311.436
INFO:tensorflow:loss = 8219.976, step = 101 (0.326 sec)
INFO:tensorflow:global_step/sec: 326.317
INFO:tensorflow:loss = 13450.36, step = 201 (0.305 sec)
INFO:tensorflow:global_step/sec: 381.471
INFO:tensorflow:loss = 11432.693, step = 301 (0.262 sec)
INFO:tensorflow:global_step/sec: 348.114
INFO:tensorflow:loss = 6782.999, step = 401 (0.288 sec)
INFO:tensorflow:global_step/sec: 281.447
INFO:tensorflow:loss = 7104.355, step = 501 (0.355 sec)
INFO:tensorflow:global_step/sec: 382.758
INFO:tensorflow:loss = 11183.514, step = 601 (0.261 sec)
INFO:tensorflow:Saving checkpoints for 608 into taxi_trained/mode

In [29]:
# Validation score

In [30]:
def print_rmse(model, df):
    metrics = model.evaluate(input_fn(df))
    print(np.sqrt(metrics['average_loss']))
  

In [34]:
print_rmse(model, df_valid)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-06-16-15:57:50
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from taxi_trained/model.ckpt-608
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-06-16-15:57:51
INFO:tensorflow:Saving dict for global step 608: average_loss = 109.30088, global_step = 608, loss = 13892.059
10.454706


In [33]:
predictions = model.predict(input_fn = predict_fn(df_valid))
for i in xrange(5):
  print(predictions.next())

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from taxi_trained/model.ckpt-608
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
{'predictions': array([10.916248], dtype=float32)}
{'predictions': array([10.919176], dtype=float32)}
{'predictions': array([10.9123745], dtype=float32)}
{'predictions': array([10.974486], dtype=float32)}
{'predictions': array([10.916514], dtype=float32)}


In [35]:
# This explains why the RMSE was so high -- the model essentially predicts the same amount for every trip. Would a more complex model help? Let's try using a deep neural network. The code to do this is quite straightforward as well.

In [36]:
# Deep Neural Network regression

In [42]:
tf.logging.set_verbosity(tf.logging.INFO)
shutil.rmtree(OUTDIR, ignore_errors = True) # start fresh each time
dn_model = tf.estimator.DNNRegressor(hidden_units = [32, 8, 2],feature_columns = features_col, model_dir = OUTDIR)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_train_distribute': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fac18181090>, '_evaluation_master': '', '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': 'taxi_trained', '_global_id_in_cluster': 0, '_save_summary_steps': 100}


In [43]:
dn_model.train(input_fn(df_train)) 

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into taxi_trained/model.ckpt.
INFO:tensorflow:loss = 24818.781, step = 1
INFO:tensorflow:global_step/sec: 294.362
INFO:tensorflow:loss = 20112.402, step = 101 (0.346 sec)
INFO:tensorflow:global_step/sec: 330.37
INFO:tensorflow:loss = 24669.14, step = 201 (0.301 sec)
INFO:tensorflow:global_step/sec: 299.615
INFO:tensorflow:loss = 24426.986, step = 301 (0.334 sec)
INFO:tensorflow:global_step/sec: 354.071
INFO:tensorflow:loss = 18019.113, step = 401 (0.282 sec)
INFO:tensorflow:global_step/sec: 341.084
INFO:tensorflow:loss = 18193.334, step = 501 (0.293 sec)
INFO:tensorflow:global_step/sec: 304.827
INFO:tensorflow:loss = 21728.312, step = 601 (0.327 sec)
INFO:tensorflow:Saving checkpoints for 608 into taxi_trained/mo

In [44]:
print_rmse(dn_model, df_valid)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-06-16-16:08:57
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from taxi_trained/model.ckpt-608
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-06-16-16:08:58
INFO:tensorflow:Saving dict for global step 608: average_loss = 196.62697, global_step = 608, loss = 24991.137
14.022374
